In [2]:
import requests
import polars as pl
from datetime import datetime, timedelta
import time
import config
API_KEY1 = config.polygon_api1
API_KEY2 = config.polygon_api2

In [3]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/sp500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])

In [4]:
API_KEY = API_KEY1
for k in range(len(stockComponents)):
    ticker = stockComponents[k]
    timespan = "day"
    start_date = datetime(2000, 1, 1)
    end_date = datetime(2023, 7, 26)
    all_data = []

    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{start_date.date()}/{end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if "results" in data:
        all_data.extend(data["results"])
    else:
        print(data)
    df = pl.DataFrame(all_data)
    df = df.rename({"v":"volume", "vw":"weightedVolume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"numberOfTrades"})
    df.write_csv('../Data/SP500/dailyHist2000/'+stockComponents[k]+'.csv')
    time.sleep(6)
    if API_KEY == API_KEY1:
        API_KEY = API_KEY2
    elif API_KEY == API_KEY2:
        API_KEY = API_KEY1

In [9]:
API_KEY = API_KEY1
for k in range(61, len(stockComponents)):
    ticker = stockComponents[k]
    timespan = "minute"
    start_date = datetime(2023, 2, 27)
    end_date = datetime(2023, 8, 14)
    batch_size = 42
    current_date = start_date
    all_data = []

    while current_date < end_date:
        batch_end_date = current_date + timedelta(days=batch_size)
        url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{current_date.date()}/{end_date.date()}?adjusted=true&sort=asc&limit=50000&apiKey={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if "results" in data:
            all_data.extend(data["results"])
        else:
            print(data)
        current_date = batch_end_date + timedelta(days=1)
    df = pl.DataFrame(all_data)
    df = df.rename({"v":"volume", "vw":"weighted volume", "o":"open", "c":"close", "h":"high", "l":"low", "t":"time", "n":"number of trades"})
    df.write_csv('../Data/SP500/minuteData/'+stockComponents[k]+'.csv')
    time.sleep(30)
    if API_KEY == API_KEY1:
        API_KEY = API_KEY2
    elif API_KEY == API_KEY2:
        API_KEY = API_KEY1

In [5]:
df = pl.DataFrame(all_data)